In [ ]:
'''
import warnings
#warnings.filterwarnings("ignore") #Evitar escribir warinings con ensembles
import pandas as pd
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

def load_experimento_4():
    # Divido los datos en data y target
    y = experimento_4_nor['Key'].values
    X = experimento_4_nor.drop(columns=['Key']).values

    return {'data': X, 'target': y}

# Defino los scorers
acc_scorer = make_scorer(accuracy_score)
bal_acc_scorer = make_scorer(balanced_accuracy_score)

# Defino los conjuntos de datos
datasets = [load_experimento_4()]
nombres = ["exp_4"]
tablas = ["Accuracy", "Balanced Accuracy"]
scoring = [acc_scorer, bal_acc_scorer]

# Lista para almacenar los DataFrames
lista = [] 

import time
inicio = time.time()

for tabla, score in zip(tablas, scoring): # Por cada tabla y su correcpondiete score
    
    dataset_names = []  # Lista para almacenar los nombres de los datasets
    knn_means = []      # Lista para almacenar las medias de KNeighborsClassifier
    tree_means = []     # Lista para almacenar las medias de DecisionTreeClassifier
    rf_means = []       # Lista para almacenar las medias de Random Forest
    
    for dataset, nombre in zip(datasets, nombres): # Por cada dataset y su correcpondiete nombre
      
        # Calculo las  metricas       
        knn = cross_val_score(KNeighborsClassifier(), X, y, cv=CV, scoring=score)      
        tree = cross_val_score(DecisionTreeClassifier(), X, y, cv=CV, scoring=score)  
        rf = cross_val_score(RandomForestClassifier(), X, y, cv=CV, scoring=score)

        # Calculo la media de las metricas anteriores
        knn_mean = knn.mean()
        tree_mean = tree.mean()
        rf_mean = rf.mean()

        # Agrego las medias de los resultados a cada lista
        dataset_names.append(nombre)
        knn_means.append(knn_mean)
        tree_means.append(tree_mean)
        rf_means.append(rf_mean)

    # Creo dataframes a partir de las listas con las medias de los resultados
    resultados = pd.DataFrame({
    'Data': dataset_names,
    'KNN': knn_means,
    'TREE': tree_means,
    'RF': rf_means,
    })

    # Agrego el dataframe creado con los resultados a la lista
    lista.append(resultados)
  
# Saco por pantalla las tablas de cada metrica
fin = time.time()
print(fin-inicio)
for index, (dataframe, tabla) in enumerate(zip(lista, tablas)):
    print(tabla)                              
    display(dataframe)
    print()
'''    

In [ ]:
#https://medium.com/@martacasdelg/c%C3%B3mo-identificar-y-tratar-outliers-con-python-bf7dd530fc3

# Calculate z-score for each data point and compute its absolute value
z_scores = zscore(experimento_4_filtrado['Delta'])
abs_z_scores = np.abs(z_scores)

# Select the outliers using a threshold of 3
outliers = experimento_4_filtrado[abs_z_scores > 3]
#print(f'Number of outliers: {len(outliers)}')

In [ ]:
### umbral Zcore https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule

### Con 3   99.7%
### Con 3.5 99.95%
### Con 4   99.993%

In [ ]:
exp_outliners = []

# Iterar sobre los dataframes y aplicar el proceso a cada uno de ellos
for exp in exp_filtrados:
    
    # Calcular el z-score para cada valor en el DataFrame
    z_scores = stats.zscore(exp)

    # Definir un umbral para los z-scores que considerarás como outliers
    umbral_zscore = 3

    # Detectar outliers basados en el z-score (valores cuyo valor absoluto de z-score es mayor que el umbral)
    outliers = (abs(z_scores) > umbral_zscore)

    # Contar el número de outliers en cada columna
    num_outliers_por_columna = outliers.sum(axis=0)

    # Obtener las filas y sus índices que son outliers en cada columna
    outliers_por_columna = {}
    for column in exp.columns:
        outliers_index = outliers[outliers[column]].index
        outliers_por_columna[column] = outliers_index

    # Imprimir el número de outliers por cada columna
    print("Número de outliers por columna:")
    print(num_outliers_por_columna)

    # Imprimir las filas con sus índices que son outliers en cada columna
    print("\nFilas con sus índices que son outliers por columna:")
    for column, indices in outliers_por_columna.items():
        print(f"Columna '{column}': {list(zip(indices, exp.loc[indices][column]))}")

    # Lista para almacenar todos los índices de outliers
    todos_indices_outliers = []

    # Almacenar todos los índices de outliers en una lista
    for indices in outliers_por_columna.values():
        todos_indices_outliers.extend(indices)

    # Convertir la lista de índices a un conjunto para eliminar duplicados
    conjunto_indices_outliers = set(todos_indices_outliers)

    # Lista ordenada de los índices únicos que no se repiten
    indices_unicos_ordenados = sorted(list(conjunto_indices_outliers))

    # Imprimir los índices únicos ordenados
    print("Índices únicos de outliers ordenados:")
    print(len(indices_unicos_ordenados))
    print(indices_unicos_ordenados)

    # Construir un nuevo DataFrame que contenga las filas correspondientes a los índices únicos ordenados
    dataframe_outliers = exp.loc[indices_unicos_ordenados]

    # Imprimir el nuevo DataFrame de filas outliers únicas
    print("Nuevo DataFrame de filas outliers únicas:")
    display(dataframe_outliers)
    
    # Agregar el DataFrame filtrado a la lista
    exp_outliners.append(indices_unicos_ordenados)



In [ ]:

exp_sin_outliners = []

# Iterar sobre los dataframes y aplicar el proceso a cada uno de ellos
for exp, out in zip(experimentos, exp_outliners) :

    exp_sin_outliner = exp.drop(out)

    exp_sin_outliner = exp_sin_outliner.reset_index(drop=True)

    exp_sin_outliners.append(exp_sin_outliner)

In [ ]:
#MinMAX

def normalizar_dataframe(dataframe):
    # Selecciono las columnas a normalizar
    columnas = ['Attention', 'Meditation', 'Delta', 'Theta', 'LowAlpha', 'HighAlpha', 'LowBeta', 'HighBeta', 'LowGamma', 'HighGamma']

    # Normalizo las columnas seleccionadas
    dataframe[columnas] = MinMaxScaler().fit_transform(dataframe[columnas])
    
    return dataframe

# Normalizar los 4 DataFrames diferentes de entrada y de salida
experimento_1_minmax = normalizar_dataframe(exp_sin_outliners[0])
experimento_2_minmax = normalizar_dataframe(exp_sin_outliners[1])
experimento_3_minmax = normalizar_dataframe(exp_sin_outliners[2])
experimento_4_minmax = normalizar_dataframe(exp_sin_outliners[3])


In [ ]:
# Lista que contendrá los DataFrames filtrados
exp_filtrados = []

# Iterar sobre los dataframes y aplicar la operación de filtrado a cada uno de ellos
for exp in experimentos:
    # Filtro el DataFrame para dejar fuera las columnas especificadas
    exp_filtrado = exp.drop(['Timestamp', 'Key'], axis=1)
    
    # Agregar el DataFrame filtrado a la lista
    exp_filtrados.append(exp_filtrado)

In [ ]:
#https://medium.com/@martacasdelg/c%C3%B3mo-identificar-y-tratar-outliers-con-python-bf7dd530fc3
#https://seaborn.pydata.org/generated/seaborn.boxplot.html
    
# Filtro el DataFrame para dejar fuera las columnas especificadas
experimento_4_filtrado = experimento_4_clean.drop(['Timestamp', 'Key'], axis=1)

# Calcul0 el número de filas necesarias para mostrar dos boxplots por fila
num_cols = len(experimento_4_filtrado.columns)
num_rows = (num_cols) // 2

# Definir el tamaño de la figura
plt.figure(figsize=(12, 6))

# Iterar sobre cada columna del DataFrame filtrado
for i, column in enumerate(experimento_4_filtrado.columns):
    # Crear un subplot para cada columna
    plt.subplot(num_rows, 2, i + 1)
    
    # Trazar un boxplot para la columna actual
    sns.boxplot(data=experimento_4_filtrado, x=column)
    
    # Establecer el título del subplot como el nombre de la columna
    plt.title(column)

# Ajustar el diseño para evitar superposiciones
plt.tight_layout()

# Mostrar el gráfico
plt.show()

In [ ]:
experimento_1_minmax.to_csv('../datos/experimento_1_nor.csv', index=False)
experimento_2_minmax.to_csv('../datos/experimento_2_nor.csv', index=False)
experimento_3_minmax.to_csv('../datos/experimento_3_nor.csv', index=False)
experimento_4_minmax.to_csv('../datos/experimento_4_nor.csv', index=False)
experimento_all_minmax = pd.concat([experimento_1_minmax, experimento_2_minmax, experimento_3_minmax, experimento_4_minmax])

# Guardar el resultado en un archivo CSV
experimento_all_minmax.to_csv('../datos/experimento_all_nor.csv', index=False) 

print()
print(color.BOLD +  color.UNDERLINE + color.RED + "Exporto todos los experimentos a un csv para tratarlo despues" + color.END )
print()
print(color.BOLD +  color.UNDERLINE + color.RED + "Exporto todos los experimentos a varios csv para tratarlos despues" + color.END )
print()


In [ ]:
experimento_4_clean = pd.read_csv('../datos/experimento_4_clean.csv', sep=',')
experimento_3_clean = pd.read_csv('../datos/experimento_3_clean.csv', sep=',')
experimento_2_clean = pd.read_csv('../datos/experimento_2_clean.csv', sep=',')
experimento_1_clean = pd.read_csv('../datos/experimento_1_clean.csv', sep=',')


experimentos = [experimento_1_clean, experimento_2_clean, experimento_3_clean, experimento_4_clean]

In [ ]:

# sacar los 4 por pantalla

data = experimento_1_minmax
# creating the dataset

#y = dataframe['Key'].values
#X = dataframe.drop(columns=['Key']).values

labels1 = {'Left': "04", 'Right': "02" ,'Up': "01", 'Down': "03",'Nothing': "00"}

#labels = set(data['Key'].values)
values = data['Key'].values

labels = data['Key']. replace(labels1)

  
fig = plt.figure(figsize = (10, 5))
 
# Calcular la cantidad de cada elemento en la columna Key
counts = data['Key'].value_counts()

# Crear el barplot
plt.bar(counts.index, counts.values, color ='maroon', width = 0.4)

# Añadir etiquetas
plt.xlabel('Elemento')
plt.ylabel('Cantidad')
plt.title('Cantidad de elementos en la columna Key')

# Mostrar el gráfico
plt.show()


In [ ]:
# Modelos
# ==============================================================================
modelo_1 = MLPClassifier(
                hidden_layer_sizes=(5),
                learning_rate_init=0.01,
                solver = 'lbfgs',
                max_iter = 1000,
                random_state = SEED
            )

modelo_2 = MLPClassifier(
                hidden_layer_sizes=(10),
                learning_rate_init=0.01,
                solver = 'lbfgs',
                max_iter = 1000,
                random_state = SEED
            )

modelo_3 = MLPClassifier(
                hidden_layer_sizes=(20, 20),
                learning_rate_init=0.01,
                solver = 'lbfgs',
                max_iter = 5000,
                random_state = SEED
            )

modelo_4 = MLPClassifier(
                hidden_layer_sizes=(50, 50, 50),
                learning_rate_init=0.01,
                solver = 'lbfgs',
                max_iter = 5000,
                random_state = SEED
            )


In [ ]:

    
y = experimento_1['Key'].values
X = experimento_1.drop(columns=['Key']).values




modelo_1.fit(X=X, y=y)

modelo_2.fit(X=X, y=y)

modelo_3.fit(X=X, y=y)

modelo_4.fit(X=X, y=y)



In [ ]:
'''modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_holdout(exp))

datos_holdout = pd.DataFrame(modelos_all, columns=['Modelo1', 'Modelo2', 'Modelo3', 'Modelo4'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_holdout = datos_holdout.rename(index=nuevos_indices)
'''

In [ ]:
'''
fig, axs = plt.subplots(2, 2, figsize=(12,8))
axs = axs.flatten()
num_points = 5  # o cualquier otro número más pequeño que 100
grid_x1 = np.linspace(start=min(X[:, 0]), stop=max(X[:, 0]), num=num_points)
grid_x2 = np.linspace(start=min(X[:, 1]), stop=max(X[:, 1]), num=num_points)
grid_x3 = np.linspace(start=min(X[:, 2]), stop=max(X[:, 2]), num=num_points)
grid_x4 = np.linspace(start=min(X[:, 3]), stop=max(X[:, 3]), num=num_points)
grid_x5 = np.linspace(start=min(X[:, 4]), stop=max(X[:, 4]), num=num_points)
grid_x6 = np.linspace(start=min(X[:, 5]), stop=max(X[:, 5]), num=num_points)
grid_x7 = np.linspace(start=min(X[:, 6]), stop=max(X[:, 6]), num=num_points)
grid_x8 = np.linspace(start=min(X[:, 7]), stop=max(X[:, 7]), num=num_points)
grid_x9 = np.linspace(start=min(X[:, 8]), stop=max(X[:, 8]), num=num_points)
grid_x10 = np.linspace(start=min(X[:, 9]), stop=max(X[:, 9]), num=num_points)

# Usa los puntos reducidos para crear el grid
xx, yy, zz, aa, bb, cc, dd, ee, ff, gg = np.meshgrid(grid_x1, grid_x2, grid_x3, grid_x4, grid_x5,
                                                      grid_x6, grid_x7, grid_x8, grid_x9, grid_x10)
X_grid = np.column_stack([xx.flatten(), yy.flatten(), zz.flatten(), aa.flatten(), bb.flatten(),
                          cc.flatten(), dd.flatten(), ee.flatten(), ff.flatten(), gg.flatten()])


for i, modelo in enumerate([modelo_1, modelo_2, modelo_3, modelo_4]):
    
    predicciones = modelo.predict(X_grid)
    
    for j in np.unique(predicciones):
        axs[i].scatter(
            x = X_grid[predicciones == j, 0],
            y = X_grid[predicciones == j, 1], 
            c = plt.rcParams['axes.prop_cycle'].by_key()['color'][j],
            #marker = 'o',
            alpha = 0.3,
            label= f"Grupo {j}"
        )

    for j in np.unique(y):
        axs[i].scatter(
            x = X[y == j, 0],
            y = X[y == j, 1], 
            c = plt.rcParams['axes.prop_cycle'].by_key()['color'][j],
            marker    = 'o',
            edgecolor = 'black'
        )
        
    axs[i].set_title(f"Capas ocultas: {modelo.hidden_layer_sizes}")
    axs[i].axis('off')
axs[0].legend();
'''

In [ ]:
'''

import pandas as pd
import numpy as np
import time
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

clasificadores = [(MLPClassifier(max_iter=2000),"MLP"),
                  (SVC(),"SVC")]

clasificadores = [(modelo_1,"Model1"),
                  (modelo_2,"Model2"),
                  (modelo_3,"Model3"),
                  (modelo_4,"Model4")]

# Numero de folds
n_folds = 2

experimentos = [("Exp1", experimento_1_nor), 
                ("Exp2", experimento_2_nor), 
                ("Exp3", experimento_3_nor), 
                ("Exp4", experimento_4_nor)]



processed_datasets = []
for name, dataset in experimentos:
    y = dataset['Key'].values
    X = dataset.drop(columns=['Timestamp','Key']).values
    processed_datasets.append((name, X, y))

#Defino los rangos de parámetros para SVM
param_grid_svm = {
    'C': np.linspace(0.01, 100000000, 10),  # Rango de valores para C
    'gamma': np.linspace(0.00000001, 1000, 10)  # Rango de valores para gamma
}

# Defino los rangos de parámetros para MLP
param_grid_mlp = {
    'alpha': np.linspace(0.00001, 0.1, 10),  # Rango de valores para alpha
    'hidden_layer_sizes': [(50,), (100,), (200,), (500,), (1000,)]  # Rango de valores para neuronas en la capa oculta
}

# Funcion para validacion cruzada
def get_cv_results(clasificador, X, y, n_folds):
    cv_scores = cross_val_score(clasificador, X, y, cv=n_folds)
    return cv_scores.mean()

# Funcion para validacion cruzada de Pipeline
def get_pipeline_results(clasificador, X, y, n_folds):
    #X = add_random_permutations(X)
    pipeline = Pipeline([
        ('feature_selection', SelectFromModel(RandomForestClassifier(random_state=SEED), threshold="median")),
        ('classification', clasificador)
    ])
    cv_scores = cross_val_score(pipeline, X, y, cv=n_folds)

    return cv_scores.mean()

# Genero las cuadriculas para a posteriori hallar los mejores parametros
grid_search_mlp = GridSearchCV((MLPClassifier(max_iter=2000)), param_grid_mlp, cv=n_folds)
grid_search_svm = GridSearchCV(SVC(), param_grid_svm, cv=n_folds)

clasificadores[0] = clasificadores[0] + (grid_search_mlp,)
clasificadores[1] = clasificadores[1] + (grid_search_svm,)                              


inicio_tiempo = time.time()

# Realizo experimentos
resultados_opt = []
comparar = np.zeros(len(experimentos)*2)
comparador = {}

for clasificador, clasi_name, clasi_opt in clasificadores:
    clasi_resultados = {}

    for dataset_name, X, y in processed_datasets:
        # Se dividen los datos. X_train_val sera el 90% de los datos y X_test el 10%
        X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=SEED, stratify=y, train_size=TRAIN)

        # Se dividen los datos. X_train sera el 90% de los datos de X_train_val y X_val el restante 10%
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=SEED, stratify=y, train_size=TRAIN)
        
       
        #Experimento 2
        clasi_opt.fit(X_train, y_train)
        print(clasi_name)
        print(clasi_opt.best_params_) #imprimo los mejores parametros solo por curiosidad
        optimized_scores = get_pipeline_results(clasi_opt.best_estimator_, X_val, y_val, n_folds)
        
        # Almaceno los resultados de los experimentos
        #clasi_resultados[dataset_name + '_Sin_Opt'] = pipe_scores
        clasi_resultados[dataset_name + '_Con_Opt'] = optimized_scores
        
    
    # Almaceno los resultados
    resultados_opt.append(clasi_resultados)

fin_tiempo = time.time()

tiempo_utilizado = fin_tiempo - inicio_tiempo

# Convierto el tiempo a horas, minutos y segundos para saber cuanto tarda la ejecucion
horas = int(tiempo_utilizado // 3600)
minutos = int((tiempo_utilizado % 3600) // 60)
segundos = int(tiempo_utilizado % 60)
print(f"Tiempo: {horas} : {minutos} : {segundos}")

# Creo el dataframe
resultados_opt_df = pd.DataFrame(resultados_opt, index=['MPL', 'SVC'])
                               
                               
#formateo las columnas e index
resultados_opt_df.columns = pd.MultiIndex.from_tuples([
                                        ("Exp1","Pipe_Opt"),
                                        ("Exp2","Pipe_Opt"),
                                        ("Exp3","Pipe_Opt"),
                                        ("Exp4","Pipe_Opt")])


display(resultados_opt_df)
'''

In [ ]:
def get_acc_results(clasificador, X, y, n_folds):
    cv_scores = accuracy_score(X, y)
    report = classification_report(X, y)
    matriz_confusion = confusion_matrix(X, y)
    
    return cv_scores.mean()


def get_cv_results(clasificador, X, y, n_folds):
    cv_scores = cross_val_score(clasificador, X, y, cv=n_folds)
    return cv_scores.mean()
